In [1]:
import sys
sys.path.append('../')


import uniflocpy.uWell.uPipe as Pipe
import uniflocpy.uTools.data_workflow as tool
import uniflocpy.uTools.uconst as uc

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools

init_notebook_mode(connected=True)

In [2]:
tube = Pipe.Pipe()

In [3]:
tube_data = tool.Data()

In [4]:
p_init_bar = 90
t_fluid_init_c = 90
t_earth_init_c = 89
q_liq_m3day = 50
watercut_perc = 20
d_tube_m = 0.065
work_time_sec = 100 * 24 * 60 * 60
geo_grad_cm = 0.027
depth_m = 3000
step_m = 10

In [5]:
tube.fluid_flow.qliq_on_surface_m3day = q_liq_m3day
tube.fluid_flow.fw_on_surface_perc = watercut_perc
tube.fluid_flow.d_m = d_tube_m

In [6]:
tube.section_casing = False

In [7]:
tube.t_earth_init_c = t_earth_init_c
tube.time_sec = work_time_sec

In [8]:
tube.calc_p_grad_pam(p_init_bar, t_fluid_init_c)

4469.775066243482

In [9]:
tube.calc_t_grad_cm(p_init_bar, t_fluid_init_c)

0.0022494241035794717

In [10]:
p_list_bar = [p_init_bar]
t_list_c = [t_fluid_init_c]
h_list_m = [depth_m]
t_earth_list_c = [t_earth_init_c]
distance_from_bottomhole = [0]

In [11]:
for i in range(int(depth_m / step_m)):
    if p_list_bar[-1] > 2:
        p_grad_pam = tube.calc_p_grad_pam(p_list_bar[-1], t_list_c[-1])
        t_grad_cm = tube.calc_t_grad_cm(p_list_bar[-1], t_list_c[-1])
        
        p_bar = p_list_bar[-1] - uc.Pa2bar(p_grad_pam) * step_m
        t_c = t_list_c[-1] - t_grad_cm * step_m
        t_earth_c = t_earth_list_c[-1] - geo_grad_cm * step_m
        h_m = h_list_m[-1] - step_m
        distance_m = distance_from_bottomhole[-1] + step_m
        
        h_list_m.append(h_m)
        p_list_bar.append(p_bar)
        t_list_c.append(t_c)
        t_earth_list_c.append(t_earth_c)
        distance_from_bottomhole.append(distance_m)
        
        tube.t_earth_init_c = t_earth_c
        

In [18]:
def plot():
    layout = dict(title =  'L(P бар, T C)',  yaxis=dict(title='Расстояние от забоя, м'))
    fig = dict(data=data, layout=layout)

    iplot(fig, filename='basic-scatter')

In [19]:
trace1 = go.Scattergl(
    x = t_earth_list_c,
    y = distance_from_bottomhole, 
    name = 'Геотерма, C',
    mode = 'markers'
)

trace2 = go.Scattergl(
    x = p_list_bar,
    y = distance_from_bottomhole, 
    name = 'КРД Беггз Брилл, бар',
    mode = 'markers'
)

trace3 = go.Scattergl(
    x = t_list_c,
    y = distance_from_bottomhole, 
    name = 'КРТ Хасан Кабир, С',
    mode = 'markers'
)


data = [ trace1, trace2, trace3]

plot()